In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
#Data
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/Headlines/'

Messages_filename='Headlines.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

Train_Messages_filename='Train_Headlines.csv'
Train_Messages_full_filename=os.path.join(Data, Train_Messages_filename)

Val_Messages_filename='Val_Headlines.csv'
Val_Messages_full_filename=os.path.join(Data, Val_Messages_filename)

Test_Messages_filename='Test_Headlines.csv'
Test_Messages_full_filename=os.path.join(Data, Test_Messages_filename)

source_field='message'
target_field='Topic' #headline

seed=42
train_test_size=0.8
train_size = 0.95

Train_Messages_filename_oabs='Train_Headlines_oabs.csv'
Train_Messages_full_filename_oabs=os.path.join(Data, Train_Messages_filename_oabs)

Val_Messages_filename_oabs='Val_Headlines_oabs.csv'
Val_Messages_full_filename_oabs=os.path.join(Data, Val_Messages_filename_oabs)

In [ ]:
# Data load
#Dataset is a mix of forum posts from differnt chapters and was shuffled when it was initially created
df = pd.read_csv(Messages_full_filename,  index_col=False)

In [ ]:
df.columns

Index(['Message_Id', 'Topic', 'message'], dtype='object')

In [ ]:
df[target_field+'_len']=df[target_field].astype(str).str.split().apply(len)
df[source_field+'_len']=df[source_field].astype(str).str.split().apply(len)

In [ ]:
#Max length of headlines
df[target_field].astype(str).str.split().apply(len).max()

23

In [ ]:
#Min length of headlines
df[target_field].astype(str).str.split().apply(len).min()

1

In [ ]:
#Average length of headlines
df[target_field].astype(str).str.split().apply(len).mean()

3.997500756691313

In [ ]:
#Max length of posts
df[source_field].astype(str).str.split().apply(len).max()

4580

In [ ]:
#Min length of posts
df[source_field].astype(str).str.split().apply(len).min()

3

In [ ]:
#Average length of posts
df[source_field].astype(str).str.split().apply(len).mean()

120.37114195529035

In [ ]:
df = df[[source_field,target_field]]
df = df.dropna(how='any',axis=0) 

In [ ]:
len(df)

115634

In [ ]:
#Make sure there is no empty headlines
df[df[target_field].isnull()].head()

,message,Topic


In [ ]:
#Make sure there is no empty texts
df[df[source_field].isnull()].head()

,message,Topic


In [ ]:
#split
train_test_records=df.sample(frac=train_test_size,random_state = seed)
val_records=df.drop(train_test_records.index).reset_index(drop=True)

train_records=train_test_records.sample(frac=train_size,random_state = seed)
test_records=train_test_records.drop(train_records.index).reset_index(drop=True)

train_records = train_records.reset_index(drop=True)

In [ ]:
len(train_records)

87882

In [ ]:
train_records.to_csv(Train_Messages_full_filename, header=True, index=False)

In [ ]:
len(val_records)

23127

In [ ]:
val_records.to_csv(Val_Messages_full_filename, header=True, index=False)

In [ ]:
len(test_records)

4625

In [ ]:
test_records.to_csv(Test_Messages_full_filename, header=True, index=False)

In [ ]:
from numpy.core.defchararray import find
from numpy.core.defchararray import replace

In [ ]:
train_records = pd.read_csv(Train_Messages_full_filename,  index_col=False)

In [ ]:
train_records['size'] = train_records[source_field].str.split().apply(len)

In [ ]:
a = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(train_records[source_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
b = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(train_records[target_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
train_records['Topic_pos']=find(a, b)

In [ ]:
def HeaderType(row,column):
  if row[column]==-1:
    return 'Abstractive'
  elif  row[column]<=row['size']/3:
    return 'From Start'
  elif  row[column]>row['size']/3 and row[column]<2*row['size']/3:
    return 'From Middle'   
  elif row[column]>=2*row['size']/3:
    return 'From End'

In [ ]:
train_records['Topic_Type'] = train_records.apply(lambda x: HeaderType(x, 'Topic_pos'), axis=1)

In [ ]:
train_records = train_records[train_records['Topic_Type']=='Abstractive']

In [ ]:
len(train_records)

78658

In [ ]:
train_records = train_records[[source_field, target_field]]

In [ ]:
train_records.to_csv(Train_Messages_full_filename_oabs, header=True, index=False)

In [ ]:
val_records = pd.read_csv(Val_Messages_full_filename,  index_col=False)

In [ ]:
val_records['size'] = val_records[source_field].str.split().apply(len)

In [ ]:
a = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(val_records[source_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
b = np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.replace(np.char.upper(val_records[target_field].values.astype(str)),'.',''),',',''),'!',''),'?',''),':',''),';','')
val_records['Topic_pos']=find(a, b)

In [ ]:
val_records['Topic_Type'] = val_records.apply(lambda x: HeaderType(x, 'Topic_pos'), axis=1)

In [ ]:
val_records = val_records[val_records['Topic_Type']=='Abstractive']

In [ ]:
len(val_records)

20809

In [ ]:
val_records = val_records[[source_field, target_field]]

In [ ]:
val_records.to_csv(Val_Messages_full_filename_oabs, header=True, index=False)